# Roommate Bill Splitting
Takes in a CSV file thas has to have the following headers, but can optionally have any others

| Amount | Purchaser | Paid |

Where "**Amount**" is a float, optionally with a $, "**Purchaser**" is a string of who made the purchase, and "**Paid**" is a string of either "Yes" or "No" which determines whether it has been paid

In [1]:
import numpy as np
import pandas as pd
import math
from itertools import combinations

### Grabbing list of roommates

In [2]:
num_rmmts = int(input("How Many Roommates/People: "))

# Generating a list of roommates
rmmts = []
for i in range(num_rmmts):
    rmmts.append(input(f"Input roommate {str(i+1)}'s name: ").lower())

How Many Roommates/People: 4
Input roommate 1's name: John
Input roommate 2's name: Adam
Input roommate 3's name: Raul
Input roommate 4's name: Stacy


### Extracting data from CSV 
Make sure this is placed in the same folder as this script

In [3]:
csv_to_read = input("Input CSV name: ")

# Importing the CSV as a Pandas DataFrame
expenses = pd.read_csv(f"{csv_to_read}.csv")

Input CSV name: expenses


In [4]:
expense_list = []

# Grabbing only the "Amount", "Purchaser", and "Paid" columns
for i in range(len(expenses)):
    expense_list.append(list(expenses.loc[i, ["Amount", "Purchaser", "Paid"]]))

### Creating payments matrix

In [5]:
# Creating a num_rmmts x num_rmmts matrix of payments
# where (i, j) represents an amount that 
# the person associated with index number j owes person i

payments_matrix = np.zeros(shape = [num_rmmts, num_rmmts])

for expense in expense_list:                              # For each expense
    if expense[2].lower() == "no":                        # if it hasn't been paid yet
        total_cost = float(expense[0].split("$")[-1])     # find its total cost
        cost = total_cost / int(num_rmmts)                # and divide it by the number of roommates
        
        try:                                              # try to 
            purchaser =  rmmts.index(expense[1].lower())  # find a roommate which matches the expense purchaser
        except Exception as e:                            # and if not
            print(f"WARNING: {e}")                        # print an error message
            break                                         # and break the code
            
        for i in range(len(payments_matrix[purchaser])):  # For each roommate
               if i != purchaser:                         # if it isn't the purchaser
                    payments_matrix[purchaser][i] += cost # add the divided cost to their debt

In [6]:
# Generating a list of pairs of roommates
pairs = list(combinations(range(num_rmmts), 2))
final_payments = np.zeros(int(num_rmmts * (num_rmmts - 1) / 2))

# Generating the final list of payments
for i, pair  in enumerate(pairs):
    final_payments[i] += -1 * payments_matrix[pair[0], pair[1]]
    final_payments[i] += payments_matrix[pair[1], pair[0]]

### Printing final payments

In [7]:
for i, payment in enumerate(final_payments):
    if payment > 0:
        print(f"{rmmts[pairs[i][0]].capitalize()} owes {rmmts[pairs[i][1]].capitalize()} ${final_payments[i]:.2f}")
    elif payment < 0:
        print(f"{rmmts[pairs[i][1]].capitalize()} owes {rmmts[pairs[i][0]].capitalize()} ${-1 * final_payments[i]:.2f}")
    else:
        print(f"{rmmts[pairs[i][1]].capitalize()} owes {rmmts[pairs[i][0]].capitalize()} nothing")

Adam owes John $1.25
John owes Raul $35.00
Stacy owes John $12.50
Adam owes Raul $36.25
Stacy owes Adam $11.25
Stacy owes Raul $47.50
